OSIsoft Cloud Services REST API

requirements:<br>
Juptyer notebook or lab<br>
Python ocs_sample_library https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/library_samples/Python3<br>
config.ini configured for your OSIsoft Cloud Services environment https://github.com/osisoft/OSI-Samples/tree/master/ocs_samples/basic_samples/SDS/Python/SDSPy/Python3<br>

objective:<br>
Create a type, stream and write stream events

In [1]:
# specify a unique prefix to create objects, suggestion use your nickname!
example_prefix = 'SDSPrefix.'

In [2]:
# setup the environment, using statements defined in a python script file
%run setup.py

Using OSIsoft OCS sample library


In [3]:
# verify we have a connection to the tenant, the tenant name should be returned
ocsClient.tenant

'efe27258-f6d5-4ea6-a001-3e4a82777710'

# Create a type definition

In [4]:
# Specify the defintion for the type that will consist of: timestamp, index and channel

# Dictionary to describe information about the type, that will be used to define the type
thetype = {'id': f'{example_prefix}bearing_channel', # unique identifier for the type
           'name': 'bearing_channel', 
           'description': 'bearing vibration signals'}

# property - timestamp
timestamp_property = SdsTypeProperty()
timestamp_property.Id = "timestamp"
timestamp_property.SdsType = SdsType.fromDictionary({"SdsTypeCode": SdsTypeCode.DateTime.value})
timestamp_property.IsKey = True

# property - index
index_property = SdsTypeProperty()
index_property.Id = "index"
index_property.SdsType = SdsType.fromDictionary({"SdsTypeCode": SdsTypeCode.Decimal.value})

# property - channel
channel_property =  SdsTypeProperty()
channel_property.Id = "channel"
channel_property.SdsType = SdsType.fromJson({"SdsTypeCode" : SdsTypeCode.Double.value })

# type defintion
bearing = SdsType()
bearing.SdsTypeCode = SdsTypeCode.Object
bearing.Id = thetype['id']
bearing.Name = thetype['name']
bearing.Description=thetype['description']
bearing.Properties = [timestamp_property,index_property,channel_property]

# Create the type
bearing_type = ocsClient.Types.getOrCreateType(namespace_id, bearing)
print(json.dumps(json.loads(bearing_type.toJson()),indent=4))

{
    "SdsTypeCode": 1,
    "Properties": [
        {
            "IsKey": true,
            "Id": "timestamp",
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 16,
                "Id": null,
                "Name": null,
                "Description": null
            },
            "Value": null,
            "Order": null
        },
        {
            "IsKey": false,
            "Id": "index",
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 15,
                "Id": null,
                "Name": null,
                "Description": null
            },
            "Value": null,
            "Order": null
        },
        {
            "IsKey": false,
            "Id": "channel",
            "Description": null,
            "SdsType": {
                "SdsTypeCode": 14,
                "Id": null,
                "Name": null,
                "Description": null
            },
            "Value": n

In [5]:
# Create 4 streams based on the created type
for bearing in range (1,5):
    stream = SdsStream(id=f'{example_prefix}Bearing{bearing}',name=f'bearing{bearing}', description=f'Bearing {bearing} Channel', typeId=f'{example_prefix}bearing_channel')
    stream = ocsClient.Streams.getOrCreateStream(namespace_id,stream)
    print(f'id: {stream.Id}, name: {stream.Name}')

id: SDSPrefix.Bearing1, name: bearing1
id: SDSPrefix.Bearing2, name: bearing2
id: SDSPrefix.Bearing3, name: bearing3
id: SDSPrefix.Bearing4, name: bearing4


In [6]:
# create stream events
values = [ {'timestamp': '2004-02-19T06:19:00Z', 'counter': 0, 'channel': -0.872},
           {'timestamp': '2004-02-19T06:21:00Z', 'counter': 1, 'channel': 0.002},
           {'timestamp': '2004-02-19T06:23:00Z', 'counter': 2, 'channel': -0.438},
           {'timestamp': '2004-02-19T06:25:00Z', 'counter': 3, 'channel': 0.222},
           {'timestamp': '2004-02-19T06:27:00Z', 'counter': 4, 'channel': 0.678}]
# insert events
for bearing in range(1,5):
        ocsClient.Streams.insertValues(namespace_id,f'{example_prefix}Bearing{bearing}',json.dumps(values))

In [7]:
# display the stream events
for stream in ocsClient.Streams.getStreams(namespace_id,query=f'{example_prefix}Bearing*'):
    value = ocsClient.Streams.getRangeValues(namespace_id,stream.Id,start="2001-01-01",count=10,value_class=None,skip=0,reverse=False,boundary_type=SdsBoundaryType.Inside)
    print(f'{stream.Id}:')
    pprint.pprint(value)

SDSPrefix.Bearing3:
[{'channel': -0.872, 'timestamp': '2004-02-19T06:19:00Z'},
 {'channel': 0.002, 'timestamp': '2004-02-19T06:21:00Z'},
 {'channel': -0.438, 'timestamp': '2004-02-19T06:23:00Z'},
 {'channel': 0.222, 'timestamp': '2004-02-19T06:25:00Z'},
 {'channel': 0.678, 'timestamp': '2004-02-19T06:27:00Z'}]
SDSPrefix.Bearing2:
[{'channel': -0.872, 'timestamp': '2004-02-19T06:19:00Z'},
 {'channel': 0.002, 'timestamp': '2004-02-19T06:21:00Z'},
 {'channel': -0.438, 'timestamp': '2004-02-19T06:23:00Z'},
 {'channel': 0.222, 'timestamp': '2004-02-19T06:25:00Z'},
 {'channel': 0.678, 'timestamp': '2004-02-19T06:27:00Z'}]
SDSPrefix.Bearing1:
[{'channel': -0.872, 'timestamp': '2004-02-19T06:19:00Z'},
 {'channel': 0.002, 'timestamp': '2004-02-19T06:21:00Z'},
 {'channel': -0.438, 'timestamp': '2004-02-19T06:23:00Z'},
 {'channel': 0.222, 'timestamp': '2004-02-19T06:25:00Z'},
 {'channel': 0.678, 'timestamp': '2004-02-19T06:27:00Z'}]
SDSPrefix.Bearing4:
[{'channel': -0.872, 'timestamp': '2004-02-1

In [8]:
# Clean-up environment - delete type and streams with {example_prefix} prefix defined earlier
# Note: don't do this if you plan to run subsequent notebooks in this "group"

def cleanup(run=False):
        for stream in ocsClient.Streams.getStreams(namespace_id,f'{example_prefix}*'):
            print(f'stream id: {stream.Id}, stream name: {stream.Name}')
            if run:
                try:
                    ocsClient.Streams.deleteStream(namespace_id,stream.Id)
                except:
                    pass
        for type in ocsClient.Types.getTypes(namespace_id,query=f'{example_prefix}*'):
            print(f'type id: {type.Id}, type name: {type.Name}')
            if run:
                try:
                    ocsClient.Types.deleteType(namespace_id,type.Id)
                    ocsClient.Types.deleteType(namespace_id,type.TypeId)
                except:
                    pass
        
# cleanup - disabled, i.e: False and commented:
#cleanup(False)

stream id: SDSPrefix.Bearing3, stream name: bearing3
stream id: SDSPrefix.Bearing2, stream name: bearing2
stream id: SDSPrefix.Bearing1, stream name: bearing1
stream id: SDSPrefix.Bearing4, stream name: bearing4
type id: SDSPrefix.bearing_channel, type name: bearing_channel
